# TetCTF 2020 - (Crypto) 2020th

Challenge's code:

```python
import random

if __name__ == '__main__':
    nIndices = 2
    indices = [int(input()) for _ in range(nIndices)]

    for i in range(2019):
        r = random.getrandbits(32)
        print(r if i in indices else 'Nope!')

    # please guess the 2020th number!
    if int(input()) == random.getrandbits(32):
        print(open('flag.txt').read())
```

There's a list of 2019 32-bit numbers but we only know 2 of them and must guess the $2020^{th}$ number for the flag. Known that python's $random$ module use $Mersenne Twister$ for generating random numbers, we just take a look at some implementation of $Mersenne Twister$ (like [this](https://github.com/tungpun/tp-cryptopals-solutions/blob/master/set%203/Challenge%2021%20-%20Implement%20the%20MT19937%20Mersenne%20Twister%20RNG/src.py)) and there are something interesting:

```python
def extract_number(self):
    if self.index == 0:
        self.generate_numbers()

    y = self.MT[self.index]
    y = y ^ (y >> 11)
    y = y ^ ((y << 7) & (0x9d2c5680))
    y = y ^ ((y << 15) & (0xefc60000))
    y = y ^ (y >> 18)

    self.index = (self.index + 1) % 624
    return y

def generate_numbers(self):
    for i in range(0, 623+1):
        y = (self.MT[i] & 0x80000000) + (self.MT[(i+1) % 624] & 0x7fffffff)  
        self.MT[i] = self.MT[(i + 397) % 624] ^ (y >> 1)
        if (y % 2) != 0: # y is odd
            self.MT[i] = self.MT[i] ^ (2567483615) # 0x9908b0df
```

The $Mersenne Twister RNG$ use a list of 624 elements $(MT)$ and each element map to an output after some bit operations, after iterated all of 624 numbers, $MT$ is renewed by $generate\_numbers$ function. The $generate_numbers$ function calculate new $MT[i]$ by old $MT[(i+1) \% 624]$ and $MT[(i + 397) \% 624]$, if we know these 2 values, we can calculate $MT[i]$.

In [1]:
import random
from sock import Sock

In [2]:
def unshiftRight(x, shift):
    res = x
    for i in range(32):
        res = x ^ res >> shift
    return res

def unshiftLeft(x, shift, mask):
    res = x
    for i in range(32):
        res = x ^ (res << shift & mask)
    return res

def untemper(v):
    """ Convert output to MT[i] """
    v = unshiftRight(v, 18)
    v = unshiftLeft(v, 15, 0xefc60000)
    v = unshiftLeft(v, 7, 0x9d2c5680)
    v = unshiftRight(v, 11)
    return v

def temper(y):
    """ Convert MT[i] to output """
    y = y ^ (y >> 11)
    y = y ^ ((y << 7) & (0x9d2c5680))
    y = y ^ ((y << 15) & (0xefc60000))
    y = y ^ (y >> 18)
    return y

def solve(a, b):
    res = []
    mt_i1, mt_i397 = untemper(a), untemper(b) # MT[i], MT[i+397]
    for msb in range(2):
        y = (msb * 0x80000000) + (mt_i1 & 0x7fffffff)
        mt_i = mt_i397 ^ (y >> 1)
        if (y % 2) != 0:
            mt_i = mt_i ^ 0x9908b0df
        res.append(temper(mt_i))
    return res

In [3]:
# test
test = []
for i in range(2020):
    r = random.getrandbits(32)
    test.append(r)

print solve(test[1396], test[1792])
print test[2019]

[3907985974L, 2900833076L]
3907985974


In [4]:
while True:
    s = Sock("207.148.119.58 6666")
    s.send_line("1396")
    s.send_line("1792")
    guess = []
    for _ in range(2019):
        a = s.read_line().strip()
        if "Nope" not in a:
            guess.append(int(a))
    res = solve(*guess)
    s.send_line(str(res[0])) # 50-50 :)
    resp = s.read_line().strip()
    if "TetCTF" in resp:
        print resp
        break
    s.close()
    sleep(1)

TetCTF{y0u_4r3_1nd33d_4_pr0ph3t}
